In [1]:
import numpy as np
from array import *
import csv

# using datetime module
import datetime;

# Binary Search
from bisect import bisect_left
from bisect import bisect_right

import matplotlib.pyplot as plt
import json

import multiprocessing as mp
from multiprocessing import Process, Queue
from multiprocessing import current_process
import queue
import threading


import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"
from pyspark.sql import SparkSession
import pyspark.pandas as ps
from pyspark.sql.functions import col
import pandas as pd

import random
import networkx as nx
from tqdm import tqdm


#https://python-louvain.readthedocs.io/en/latest/api.html
#community.modularity(partition, graph, weight='weight')
from community import modularity

import pickle

import powerlaw

print('----------------------')
#unique_payment_addresses_len = len(unique_payment_addresses)
unique_raw_addresses_len        = 40330345
unique_payment_addresses_len    = 40324960
unique_delegation_addresses_len = 3868049
print('unique_raw_addresses_len        = ', unique_raw_addresses_len)
print('unique_payment_addresses_len    = ', unique_payment_addresses_len)
print('unique_delegation_addresses_len = ', unique_delegation_addresses_len)

INITIAL_DATE_CARDANO      = datetime.datetime.strptime('2017-09-23 21:44:51', '%Y-%m-%d %H:%M:%S').date()
FINAL_DATE_CARDANO        = datetime.datetime.strptime('2023-01-21 17:39:30', '%Y-%m-%d %H:%M:%S').date()
total_time_length_CARDANO = int((FINAL_DATE_CARDANO - INITIAL_DATE_CARDANO).total_seconds()/86400) + 1


print('----------------------')
print('done!')



----------------------
unique_raw_addresses_len        =  40330345
unique_payment_addresses_len    =  40324960
unique_delegation_addresses_len =  3868049
----------------------
done!


In [2]:
print('----------------------')

#print(os.path.basename("/path/to/some/file.txt"))
#print(os.path.dirname("/path/to/some/file.txt"))

BASE_ADDRESS = '/local/scratch/exported/Cardano_MCH_2023_1/'
TEMP_ADDRESS = BASE_ADDRESS + '/temp_files/'


print('----------------------')
print('done!')


----------------------
----------------------
done!


In [3]:
# Define required methods:


print('----------------------')

# ct stores current time
ct = datetime.datetime.now()
print("current time: ", ct)



##########################################################################################
def parent (id1, parents_array):
    return parents_array[id1];

##########################################################################################
def find_parent (id1, parents_array):
    while (id1 != parent(id1, parents_array)):
        new_parent = parent(parent(id1, parents_array), parents_array);
        id1        = new_parent;
    
    return id1

##########################################################################################
# Link two addresses based on "Union-Find" Algorithm:
def link_address (addr_position_1, addr_position_2, parents_array):
    id1 = find_parent(addr_position_1, parents_array)
    id2 = find_parent(addr_position_2, parents_array)

    if (id1 == id2):
        return

    # make sure always we have id1 > id2
    if id1 < id2:
        x = id1
        id1 = id2
        id2 = x

    parents_array[id1] = id2
    return

##########################################################################################
def resolveAll (parents_array):
    for id1 in tqdm(range(len(parents_array))):
        parents_array[id1] = find_parent(id1, parents_array);
    return

##########################################################################################
def remapClusterIds (parents_array, clustering_array):
    cluster_count = 0
    place_holder = 9999999999999
    new_cluster_ids = [place_holder] * len(parents_array)
    
    for i in range(len(clustering_array)):
        clustering_array[i] = parents_array[i]

    for i in tqdm(range(len(clustering_array))):
        parent_index = clustering_array[i]

        if (new_cluster_ids [parent_index] == place_holder):
            new_cluster_ids [parent_index] = cluster_count
            cluster_count = cluster_count + 1

        clustering_array[i] = new_cluster_ids [parent_index]

    return cluster_count;

##########################################################################################
def merge_parents(parents_array, parents_merged):
    if (len(parents_array) != len(parents_merged)):
        print('parents_merged Error: -1 (Length)')
        return -1
    
    for i in tqdm(range(len(parents_merged))):
        link_address (i, parents_array[i], parents_merged)


##########################################################################################
def BinarySearch(a, x):
    i = bisect_left(a, x)
    if i < len(a) and a[i] == x:
        return i
    else:
        print('BinarySearch Error: -1')
        return -1

##########################################################################################
def BinarySearch_Find_start_end(a, x):
    i = bisect_left(a, x)
    j = bisect_right(a, x) - 1
    if i < len(a) and a[i] == x and j < len(a) and a[j] == x:
        return [i, j]
    else:
        print('BinarySearch Error: -1')
        print('i = ', i)
        print('j = ', j)
        return -1


##########################################################################################
def store_array_to_file (input_array_name, file_name, index_=False, header_=None):
    ct = datetime.datetime.now()
    curr_timestamp = str(ct)[0:10] + '_' + str(ct)[11:13] + str(ct)[14:16] + str(ct)[17:19]
    print('start time (Store Array to ' + file_name + '): ', ct)

    df = pd.DataFrame(input_array_name)
    df.to_csv(file_name, index=index_, header=header_)

    '''
    with open(file_name, 'w') as filehandle:
        for element in input_array_name:
            filehandle.write(f'{element}\n')
    '''

    et = datetime.datetime.now() - ct
    print('elapsed time (Store Array to ' + file_name + '): ', et)

    return

##########################################################################################
def load_file_to_array (file_name, header_=None):
    ct = datetime.datetime.now()
    curr_timestamp = str(ct)[0:10] + '_' + str(ct)[11:13] + str(ct)[14:16] + str(ct)[17:19]
    print('start time (Load ' + file_name  + ' to Array): ', ct)

    df = pd.read_csv(file_name, header=header_)
    output_array_name = df.to_numpy()
    
    '''
    output_array_name = []
    with open(file_name, 'r') as filehandle:
        for line in filehandle:
            # Remove linebreak which is the last character of the string
            curr_place = line[:-1]
            # Add item to the list
            output_array_name.append(curr_place)
    '''    

    '''
    for index, row in df.iterrows():
        new_line = df.loc[index , 0]
        output_array_name.append(new_line)
        if (index%1000000 == 0):
                print('One New Milion Records loaded: ', index)
    '''

    et = datetime.datetime.now() - ct
    print('elapsed time (Load ' + file_name  + ' to Array): ', et)
    
    return output_array_name

##########################################################################################
def store_array_to_file_2D (input_array_name, file_name):
    ct = datetime.datetime.now()
    curr_timestamp = str(ct)[0:10] + '_' + str(ct)[11:13] + str(ct)[14:16] + str(ct)[17:19]
    print('start time (Store Array 2D to ' + file_name + '): ', ct)

    with open(file_name, "w") as filehandle:
        json.dump(input_array_name, filehandle)
    
    et = datetime.datetime.now() - ct
    print('elapsed time (Store Array 2D to ' + file_name + '): ', et)

    return

##########################################################################################
def load_file_to_array_2D (file_name):
    ct = datetime.datetime.now()
    curr_timestamp = str(ct)[0:10] + '_' + str(ct)[11:13] + str(ct)[14:16] + str(ct)[17:19]
    print('start time (Load ' + file_name  + ' to Array 2D): ', ct)

    with open(file_name) as filehandle:
        output_array_name = json.load(filehandle)

    et = datetime.datetime.now() - ct
    print('elapsed time (Load ' + file_name  + ' to Array 2D): ', et)
    
    return output_array_name


##########################################################################################
def store_dict_to_file_INT (input_dict_name, file_name):
    ct = datetime.datetime.now()
    curr_timestamp = str(ct)[0:10] + '_' + str(ct)[11:13] + str(ct)[14:16] + str(ct)[17:19]
    print('start time (Store Dictionary to ' + file_name + '): ', ct)

    filehandle = csv.writer(open(file_name, 'w'))
    for key, val in input_dict_name.items():
        filehandle.writerow([key, val])

    et = datetime.datetime.now() - ct
    print('elapsed time (Store Dictionary to ' + file_name + '): ', et)

    return

##########################################################################################
def load_file_to_dict_INT (file_name):
    ct = datetime.datetime.now()
    curr_timestamp = str(ct)[0:10] + '_' + str(ct)[11:13] + str(ct)[14:16] + str(ct)[17:19]
    print('start time (Load ' + file_name  + ' to Dictionary): ', ct)

    filehandle = csv.reader(open(file_name, 'r'))
    output_dict_name = {int(rows[0]):int(rows[1]) for rows in filehandle}

    et = datetime.datetime.now() - ct
    print('elapsed time (Load ' + file_name  + ' to Dictionary): ', et)
    
    return output_dict_name

##########################################################################################
def add_edge_info(node_1, node_2, edges_array, weight=1):
    if (node_1 == node_2):
        return

    # make sure always we have n1 > n2:
    if (node_1 < node_2):
        n1 = node_2
        n2 = node_1
    else:
        n1 = node_1
        n2 = node_2

    for i in range(weight):
        edges_array[n1].append(n2)

    return

##########################################################################################
def extract_payment_delegation_parts(address_raw, payment_cred, stake_address):
    if (address_raw == ''):
        #print(' - Error: address_raw is empty!')
        return ['', '']

    if (address_raw[2] == '8'): #Byron Address
        if (payment_cred != ''):
            print(' - Error: payment_cred in Byron Address is NOT empty!')
            return ['', '']
        if (stake_address != ''):
            print(' - Error: stake_address in Byron Address is NOT empty!')
            return ['', '']
        payment_part    = address_raw
        delegation_part = ''

    else: #Shelley Address
        if (payment_cred == ''):
            print(' - Error: payment_cred in Shelley Address is empty!')
            return ['', '']
        payment_part    = payment_cred
        delegation_part = stake_address

    return [payment_part, delegation_part]

##########################################################################################
# Function which calculates the Gini index
# Inputs: array with length equal to number of agents; each element in array represents the wealth of the agent
# Outputs: gini index of the system
def gini_index(inp_array):
    array = np.array(inp_array)
    array = array.astype(float)
    array = array.flatten() #all values are treated equally, arrays must be 1d
    if np.amin(array) < 0:
        array -= np.amin(array) #values cannot be negative
    array += 0.0000001 #values cannot be 0
    array = np.sort(array) #values must be sorted
    index = np.arange(1,array.shape[0]+1) #index per array element
    n = array.shape[0] #number of array elements
    return ((np.sum((2 * index - n  - 1) * array)) / (n * np.sum(array))) #Gini coefficient

##########################################################################################



##########################################################################################



##########################################################################################
print('----------------------')
print('done!')



----------------------
current time:  2024-02-17 17:08:11.092924
----------------------
done!


In [4]:
# Read ("sorted" "unique" array_list) [raw_address_list/payment_address_list/delegation_address_list] from file:

print('----------------------')


file_name = BASE_ADDRESS + '/Unique_AddressesListRaw__Cardano_TXs_All__2023-02-28_143357.txt'
unique_raw_addresses = load_file_to_array (file_name)
print('Length of \"unique_raw_addresses\" = ' + str(len(unique_raw_addresses)))


file_name = BASE_ADDRESS + '/Unique_AddressesListPayment__Cardano_TXs_All__2023-02-28_143953.txt'
unique_payment_addresses = load_file_to_array (file_name)
print('Length of \"unique_payment_addresses\" = ' + str(len(unique_payment_addresses)))


file_name = BASE_ADDRESS + '/Unique_AddressesListDelegation__Cardano_TXs_All__2023-02-28_144415.txt'
unique_delegation_addresses = load_file_to_array (file_name)
print('Length of \"unique_delegation_addresses\" = ' + str(len(unique_delegation_addresses)))


##########################################################################################
print('----------------------')
print('done!')



----------------------
start time (Load /local/scratch/exported/Cardano_MCH_2023_1//Unique_AddressesListRaw__Cardano_TXs_All__2023-02-28_143357.txt to Array):  2024-02-17 17:08:14.520066
elapsed time (Load /local/scratch/exported/Cardano_MCH_2023_1//Unique_AddressesListRaw__Cardano_TXs_All__2023-02-28_143357.txt to Array):  0:00:47.484417
Length of "unique_raw_addresses" = 40330345
start time (Load /local/scratch/exported/Cardano_MCH_2023_1//Unique_AddressesListPayment__Cardano_TXs_All__2023-02-28_143953.txt to Array):  2024-02-17 17:09:02.005003
elapsed time (Load /local/scratch/exported/Cardano_MCH_2023_1//Unique_AddressesListPayment__Cardano_TXs_All__2023-02-28_143953.txt to Array):  0:00:36.294476
Length of "unique_payment_addresses" = 40324960
start time (Load /local/scratch/exported/Cardano_MCH_2023_1//Unique_AddressesListDelegation__Cardano_TXs_All__2023-02-28_144415.txt to Array):  2024-02-17 17:09:38.299947
elapsed time (Load /local/scratch/exported/Cardano_MCH_2023_1//Unique_

In [5]:
# Read clustering_array[] from file:

file_name = BASE_ADDRESS + '/clusteringArrayList_Heuristic1noSC__Cardano_TXs_All__2023-02-25_223957.txt'
clustering_array_heur1 = load_file_to_array (file_name)


file_name = BASE_ADDRESS + '/clusteringArrayList_Heuristic2__Cardano_TXs_All__2023-03-26_110150.txt'
clustering_array_heur2 = load_file_to_array (file_name)


file_name = BASE_ADDRESS + '/clusteringArrayList_Heuristic1noSC_AND_Heuristic2__Cardano_TXs_All__2023-03-26_141212.txt'
clustering_array_heur1and2 = load_file_to_array (file_name)


##########################################################################################
print('----------------------')
print('done!')



start time (Load /local/scratch/exported/Cardano_MCH_2023_1//clusteringArrayList_Heuristic1noSC__Cardano_TXs_All__2023-02-25_223957.txt to Array):  2024-02-17 17:09:41.101550
elapsed time (Load /local/scratch/exported/Cardano_MCH_2023_1//clusteringArrayList_Heuristic1noSC__Cardano_TXs_All__2023-02-25_223957.txt to Array):  0:00:02.703545
start time (Load /local/scratch/exported/Cardano_MCH_2023_1//clusteringArrayList_Heuristic2__Cardano_TXs_All__2023-03-26_110150.txt to Array):  2024-02-17 17:09:43.805302
elapsed time (Load /local/scratch/exported/Cardano_MCH_2023_1//clusteringArrayList_Heuristic2__Cardano_TXs_All__2023-03-26_110150.txt to Array):  0:00:02.702376
start time (Load /local/scratch/exported/Cardano_MCH_2023_1//clusteringArrayList_Heuristic1noSC_AND_Heuristic2__Cardano_TXs_All__2023-03-26_141212.txt to Array):  2024-02-17 17:09:46.507906
elapsed time (Load /local/scratch/exported/Cardano_MCH_2023_1//clusteringArrayList_Heuristic1noSC_AND_Heuristic2__Cardano_TXs_All__2023-03

In [6]:
##########################################################################################
print('----------------------')
print('\"unique_raw_addresses_len\"             = ', unique_raw_addresses_len)
print('\"unique_payment_addresses_len\"         = ', unique_payment_addresses_len)
print('\"unique_delegation_addresses_len\"      = ', unique_delegation_addresses_len)

print('Length of \"clustering_array_heur1\"     = ', len(clustering_array_heur1))
print('number of clusters_heur1               = ', len(np.unique(clustering_array_heur1)))
print('Length of \"clustering_array_heur2\"     = ', len(clustering_array_heur2))
print('number of clusters_heur2               = ', len(np.unique(clustering_array_heur2)))
print('Length of \"clustering_array_heur1and2\" = ', len(clustering_array_heur1and2))
print('number of clusters_heur1and2           = ', len(np.unique(clustering_array_heur1and2)))


##########################################################################################
print('----------------------')
print('done!')


----------------------
"unique_raw_addresses_len"             =  40330345
"unique_payment_addresses_len"         =  40324960
"unique_delegation_addresses_len"      =  3868049
Length of "clustering_array_heur1"     =  40324960
number of clusters_heur1               =  19249106
Length of "clustering_array_heur2"     =  40324960
number of clusters_heur2               =  18529342
Length of "clustering_array_heur1and2" =  40324960
number of clusters_heur1and2           =  8805791
----------------------
done!


In [ ]:
# Calculate Entities Balances:

'''
print('----------------------')
import random
import pickle


# ct stores current time
ct = datetime.datetime.now()
print("current time: ", ct)
print('----------------------')


INITIAL_DATE_CARDANO      = datetime.datetime.strptime('2017-09-23 21:44:51', '%Y-%m-%d %H:%M:%S').date()
FINAL_DATE_CARDANO        = datetime.datetime.strptime('2023-01-21 17:39:30', '%Y-%m-%d %H:%M:%S').date()
total_time_length_CARDANO = int((FINAL_DATE_CARDANO - INITIAL_DATE_CARDANO).total_seconds()/86400) + 1



# Choose from: "clustering_array_heur1"  OR  "clustering_array_heur2"  OR  "clustering_array_heur1and2"
clustering_array = clustering_array_heur1and2

balances_per_entity_array = [0] * ( np.amax(clustering_array)+1 )
current_delta_day = 0



CSV_FILES_NAME_FORMAT = BASE_ADDRESS + '/cardano_TXs_Velocity_'
NUMBER_OF_CSV_FILES = 6
CSV_FILES_SUFFIX = '.csv'


for i in range(1, NUMBER_OF_CSV_FILES + 1):

    ct_temp = datetime.datetime.now()

    file_name = CSV_FILES_NAME_FORMAT + str(i) + CSV_FILES_SUFFIX
    df = pd.read_csv(file_name, delimiter='|')

    et_temp = datetime.datetime.now() - ct_temp
    print("elapsed time (Load CSV File " + file_name + "): ", et_temp)

    ct_temp = datetime.datetime.now()

    for index, row in tqdm(df.iterrows()):
        ##########################################################################################
        TX_ID      = df.loc[index , 'TX_ID']
        ##########################################################################################
        BLOCK_TIME = datetime.datetime.strptime(str(df.loc[index , 'BLOCK_TIME']), '%Y-%m-%d %H:%M:%S').date()
        tx_delta_day = int((BLOCK_TIME - INITIAL_DATE_CARDANO).total_seconds()/86400)
        if(current_delta_day < tx_delta_day):
            output_filename = BASE_ADDRESS + '/YuZhang_Cardano_Balances_Entities/YuZhang__BalancesPerEntityDay_' + str(current_delta_day).zfill(4) + '__Cardano_TXs_All.txt'
            #print('output_filename = ', output_filename)
            #pickle.dump(balances_per_entity_array, open(output_filename, 'wb'))
            store_array_to_file(balances_per_entity_array, output_filename)
            current_delta_day = tx_delta_day

        ##########################################################################################
        EPOCH_NO   = str( df.loc[index , 'EPOCH_NO'] )
        ##########################################################################################
        inputs_list = list( df.loc[index , 'INPUTs'].split(';') )
        #for tx_input in inputs_list:
        #    address_raw           = tx_input.split(',')[4]
        #    address_has_script    = tx_input.split(',')[7]
        #    payment_cred          = tx_input.split(',')[8]
        #    stake_address         = tx_input.split(',')[9]
        #    input_value           = int(tx_input.split(',')[6])
        #    input_time_str        = tx_input.split(',')[10]
        #    INPUT_TIME            = datetime.datetime.strptime(input_time_str, '%Y-%m-%d %H:%M:%S').date()
        #    INPUT_HOLDING_DAY     = int((BLOCK_TIME - INPUT_TIME).total_seconds()/86400)
        #    
        #    # Generate a random float between 0 and 1
        #    random_float = random.random()
        #    if(random_float <= 1):
        #        hodling_day_array[INPUT_HOLDING_DAY] = hodling_day_array[INPUT_HOLDING_DAY] + input_value
        ##########################################################################################
        outputs_list = list( df.loc[index , 'OUTPUTs'].split(';') )
        #for tx_output in outputs_list:
        #    address_raw        = tx_output.split(',')[1]
        #    address_has_script = tx_output.split(',')[4]
        #    payment_cred       = tx_output.split(',')[5]
        #    stake_address      = tx_output.split(',')[6]
        #    [address_payment_part, address_delegation_part] = extract_payment_delegation_parts(address_raw, payment_cred, stake_address)
        #    if (address_payment_part != '' and address_delegation_part != ''):
        #        indx1 = BinarySearch(unique_delegation_addresses, address_delegation_part)
        #        indx2 = BinarySearch(unique_payment_addresses, address_payment_part)
        #        stake_delegation_array[indx1].append(indx2)
        ##########################################################################################
        # Update all ADA balances:
        for i in range(0, len(inputs_list)):
            address_has_script = inputs_list[i].split(',')[7]
            if (address_has_script == 'f'): # non-Smart Contract Address
                address_raw   = inputs_list[i].split(',')[4]
                payment_cred  = inputs_list[i].split(',')[8]
                stake_address = inputs_list[i].split(',')[9]
                [address_payment_part, address_delegation_part] = extract_payment_delegation_parts(address_raw, payment_cred, stake_address)
                if (address_payment_part != ''):
                    addr_indx   = BinarySearch(unique_payment_addresses, address_payment_part)
                    entity_indx = clustering_array[addr_indx][0]
                    UTXO_value  = int(inputs_list[i].split(',')[6])
                    balances_per_entity_array[entity_indx] = balances_per_entity_array[entity_indx] - int(UTXO_value)

        for i in range(0, len(outputs_list)):
            address_has_script = outputs_list[i].split(',')[4]
            if (address_has_script == 'f'): # non-Smart Contract Address
                address_raw   = outputs_list[i].split(',')[1]
                payment_cred  = outputs_list[i].split(',')[5]
                stake_address = outputs_list[i].split(',')[6]
                [address_payment_part, address_delegation_part] = extract_payment_delegation_parts(address_raw, payment_cred, stake_address)
                if (address_payment_part != ''):
                    addr_indx   = BinarySearch(unique_payment_addresses, address_payment_part)
                    entity_indx = clustering_array[addr_indx][0]
                    UTXO_value  = int(outputs_list[i].split(',')[3])
                    balances_per_entity_array[entity_indx] = balances_per_entity_array[entity_indx] + int(UTXO_value)

        ##########################################################################################

    et_temp = datetime.datetime.now() - ct_temp
    print("elapsed time (ADA Velocity from CSV File " + file_name + "): ", et_temp)



print('----------------------')
et = datetime.datetime.now() - ct
print("Total elapsed time (ADA Velocity): ", et)


print('----------------------')
print('done!')
'''





In [ ]:
# Calculate Entities TX_vol:


print('----------------------')
import random
import pickle


# ct stores current time
ct = datetime.datetime.now()
print("current time: ", ct)
print('----------------------')


INITIAL_DATE_CARDANO      = datetime.datetime.strptime('2017-09-23 21:44:51', '%Y-%m-%d %H:%M:%S').date()
FINAL_DATE_CARDANO        = datetime.datetime.strptime('2023-01-21 17:39:30', '%Y-%m-%d %H:%M:%S').date()
total_time_length_CARDANO = int((FINAL_DATE_CARDANO - INITIAL_DATE_CARDANO).total_seconds()/86400) + 1



# Choose from: "clustering_array_heur1"  OR  "clustering_array_heur2"  OR  "clustering_array_heur1and2"
clustering_array = clustering_array_heur1and2

TX_vol_per_entity_array = [0] * ( np.amax(clustering_array)+1 )
current_delta_day = 0



CSV_FILES_NAME_FORMAT = BASE_ADDRESS + '/cardano_TXs_Velocity_'
NUMBER_OF_CSV_FILES = 6
CSV_FILES_SUFFIX = '.csv'


for i in range(1, NUMBER_OF_CSV_FILES + 1):

    ct_temp = datetime.datetime.now()

    file_name = CSV_FILES_NAME_FORMAT + str(i) + CSV_FILES_SUFFIX
    df = pd.read_csv(file_name, delimiter='|')

    et_temp = datetime.datetime.now() - ct_temp
    print("elapsed time (Load CSV File " + file_name + "): ", et_temp)

    ct_temp = datetime.datetime.now()

    for index, row in tqdm(df.iterrows()):
        ##########################################################################################
        TX_ID      = df.loc[index , 'TX_ID']
        ##########################################################################################
        BLOCK_TIME = datetime.datetime.strptime(str(df.loc[index , 'BLOCK_TIME']), '%Y-%m-%d %H:%M:%S').date()
        tx_delta_day = int((BLOCK_TIME - INITIAL_DATE_CARDANO).total_seconds()/86400)
        
        if(current_delta_day < tx_delta_day):
            output_filename = BASE_ADDRESS + '/YuZhang_Cardano_TX_Vols_Entities__PICKLE/TX_Vol_PerEntityDay_' + str(current_delta_day).zfill(4) + '__Cardano_TXs_All.pickle'
            #print('output_filename = ', output_filename)

            pickle.dump(TX_vol_per_entity_array, open(output_filename, 'wb'))
            #store_array_to_file(TX_vol_per_entity_array, output_filename)
            
            TX_vol_per_entity_array = [0] * ( np.amax(clustering_array)+1 )

            current_delta_day = tx_delta_day

        ##########################################################################################
        EPOCH_NO   = str( df.loc[index , 'EPOCH_NO'] )
        ##########################################################################################
        inputs_list = list( df.loc[index , 'INPUTs'].split(';') )
        #for tx_input in inputs_list:
        #    address_raw           = tx_input.split(',')[4]
        #    address_has_script    = tx_input.split(',')[7]
        #    payment_cred          = tx_input.split(',')[8]
        #    stake_address         = tx_input.split(',')[9]
        #    input_value           = int(tx_input.split(',')[6])
        #    input_time_str        = tx_input.split(',')[10]
        #    INPUT_TIME            = datetime.datetime.strptime(input_time_str, '%Y-%m-%d %H:%M:%S').date()
        #    INPUT_HOLDING_DAY     = int((BLOCK_TIME - INPUT_TIME).total_seconds()/86400)
        #    
        #    # Generate a random float between 0 and 1
        #    random_float = random.random()
        #    if(random_float <= 1):
        #        hodling_day_array[INPUT_HOLDING_DAY] = hodling_day_array[INPUT_HOLDING_DAY] + input_value
        ##########################################################################################
        outputs_list = list( df.loc[index , 'OUTPUTs'].split(';') )
        #for tx_output in outputs_list:
        #    address_raw        = tx_output.split(',')[1]
        #    address_has_script = tx_output.split(',')[4]
        #    payment_cred       = tx_output.split(',')[5]
        #    stake_address      = tx_output.split(',')[6]
        #    [address_payment_part, address_delegation_part] = extract_payment_delegation_parts(address_raw, payment_cred, stake_address)
        #    if (address_payment_part != '' and address_delegation_part != ''):
        #        indx1 = BinarySearch(unique_delegation_addresses, address_delegation_part)
        #        indx2 = BinarySearch(unique_payment_addresses, address_payment_part)
        #        stake_delegation_array[indx1].append(indx2)
        ##########################################################################################

        # key: entity_indx | value: TX_vol
        TX_vol_dict = {}
        
        for tx_input in inputs_list:
            tx_input_split = tx_input.split(',')
            address_has_script = tx_input_split[7]
            if (address_has_script == 'f'): # non-Smart Contract Address
                address_raw   = tx_input_split[4]
                payment_cred  = tx_input_split[8]
                stake_address = tx_input_split[9]
                [address_payment_part, address_delegation_part] = extract_payment_delegation_parts(address_raw, payment_cred, stake_address)
                if (address_payment_part != ''):
                    addr_indx   = BinarySearch(unique_payment_addresses, address_payment_part)
                    entity_indx = clustering_array[addr_indx][0]
                    UTXO_value  = int(tx_input_split[6])
                    #balances_per_entity_array[entity_indx] = balances_per_entity_array[entity_indx] - int(UTXO_value)
                    TX_vol_dict[entity_indx] = TX_vol_dict.get(entity_indx, 0) - UTXO_value

        for tx_output in outputs_list:
            tx_output_split = tx_output.split(',')
            address_has_script = tx_output_split[4]
            if (address_has_script == 'f'): # non-Smart Contract Address
                address_raw   = tx_output_split[1]
                payment_cred  = tx_output_split[5]
                stake_address = tx_output_split[6]
                [address_payment_part, address_delegation_part] = extract_payment_delegation_parts(address_raw, payment_cred, stake_address)
                if (address_payment_part != ''):
                    addr_indx   = BinarySearch(unique_payment_addresses, address_payment_part)
                    entity_indx = clustering_array[addr_indx][0]
                    UTXO_value  = int(tx_output_split[3])
                    #balances_per_entity_array[entity_indx] = balances_per_entity_array[entity_indx] + int(UTXO_value)
                    TX_vol_dict[entity_indx] = TX_vol_dict.get(entity_indx, 0) + UTXO_value

        for key in TX_vol_dict:
            TX_vol_per_entity_array[key] = TX_vol_per_entity_array[key] + abs(TX_vol_dict[key])


        ##########################################################################################

    et_temp = datetime.datetime.now() - ct_temp
    print("elapsed time (Entities TX_vol from CSV File " + file_name + "): ", et_temp)



print('----------------------')
et = datetime.datetime.now() - ct
print("Total elapsed time (Entities TX_vol): ", et)


print('----------------------')
print('done!')




----------------------
current time:  2024-02-17 18:44:27.006859
----------------------
elapsed time (Load CSV File /local/scratch/exported/Cardano_MCH_2023_1//cardano_TXs_Velocity_1.csv):  0:01:19.366201


10000000it [1:35:05, 1752.64it/s]


elapsed time (Entities TX_vol from CSV File /local/scratch/exported/Cardano_MCH_2023_1//cardano_TXs_Velocity_1.csv):  1:35:05.693770
elapsed time (Load CSV File /local/scratch/exported/Cardano_MCH_2023_1//cardano_TXs_Velocity_2.csv):  0:01:23.925796


10000000it [1:30:42, 1837.26it/s]


elapsed time (Entities TX_vol from CSV File /local/scratch/exported/Cardano_MCH_2023_1//cardano_TXs_Velocity_2.csv):  1:30:42.901717
elapsed time (Load CSV File /local/scratch/exported/Cardano_MCH_2023_1//cardano_TXs_Velocity_3.csv):  0:01:38.098220


8664589it [1:24:06, 1793.86it/s]